_________________________________________________________________________________________________________________
# Project: OpenStreetMap Data Wrangling 
__________________________________________________________________________________________________________________

### Map Area : Singapore

- https://mapzen.com/data/metro-extracts/metro/singapore/  
 This map is the area of Singapore. I would like to explore this extract a little bit and see what interesting data I can find.

## Problems Encountered in the Map

## 3. Data Overview

## 4.  Additional Ideas

## 5. Conclusion

## 6. References